In [1]:
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import pandas as pd

In [2]:
train_data = pd.read_csv("../train.csv")

In [3]:
train_data.dropna(subset=['gt_25k_p6', 'md_earn_wne_p10'])

,INSTNM,CITY,STABBR,PREDDEG,CONTROL,LOCALE,SATVRMID,SATMTMID,SATWRMID,ACTCMMID,...,PCTFLOAN,UG25abv,GRAD_DEBT_MDN_SUPP,GRAD_DEBT_MDN10YR_SUPP,RPY_3YR_RT_SUPP,C150_4_POOLED_SUPP,C200_L4_POOLED_SUPP,md_earn_wne_p10,gt_25k_p6,gt_25k_05_p6
0,Kent State University at Tuscarawas,New Philadelphia,OH,2,1,32.0,NaN,NaN,NaN,NaN,...,0.6196,0.3453,26225.0,291.151261,0.690392,0.191335,NaN,36600.0,0.589350,1.0
1,McDaniel College,Westminster,MD,3,2,23.0,550.0,555.0,NaN,24.0,...,0.6629,0.0313,27000.0,299.755350,0.949511,0.725667,NaN,44600.0,0.725173,1.0
2,York Technical College,Rock Hill,SC,2,1,13.0,NaN,NaN,NaN,NaN,...,0.2122,0.4024,10475.0,116.293974,0.569444,NaN,0.183209,25300.0,0.421260,0.0
3,Career Training Academy-New Kensington,New Kensington,PA,1,3,21.0,NaN,NaN,NaN,NaN,...,0.6386,0.4540,9500.0,105.469475,0.672297,NaN,0.786278,23400.0,0.297398,0.0
5,Fort Scott Community College,Fort Scott,KS,1,1,33.0,NaN,NaN,NaN,NaN,...,0.4036,0.2206,NaN,NaN,0.506419,NaN,0.430146,26900.0,0.471015,0.0
6,University of Wisconsin-River Falls,River Falls,WI,3,1,32.0,NaN,NaN,NaN,22.0,...,0.6503,0.1126,24700.0,274.220635,0.915487,0.527869,NaN,39700.0,0.674946,1.0
7,Illinois Center for Broadcasting,Lombard,IL,1,3,21.0,NaN,NaN,NaN,NaN,...,0.8797,0.5118,9500.0,105.469475,0.676955,NaN,0.832131,29400.0,0.520619,1.0
8,University of Phoenix-Pittsburgh Campus,Pittsburgh,PA,3,3,21.0,NaN,NaN,NaN,NaN,...,0.6814,0.9189,35500.0,394.122775,0.406534,0.101329,NaN,53400.0,0.705231,1.0
11,John Tyler Community College,Chester,VA,2,1,21.0,NaN,NaN,NaN,NaN,...,0.1407,0.2917,11699.0,129.882883,0.618397,NaN,0.154572,33800.0,0.544224,1.0
13,Helene Fuld College of Nursing,New York,NY,2,2,11.0,NaN,NaN,NaN,NaN,...,0.5815,0.9621,13750.0,152.653188,0.777409,0.617650,NaN,82100.0,0.934211,1.0


In [4]:
train_data.columns

Index(['INSTNM', 'CITY', 'STABBR', 'PREDDEG', 'CONTROL', 'LOCALE', 'SATVRMID',
       'SATMTMID', 'SATWRMID', 'ACTCMMID', 'ACTENMID', 'ACTMTMID', 'ACTWRMID',
       'SAT_AVG', 'DISTANCEONLY', 'UGDS', 'UGDS_WHITE', 'UGDS_BLACK',
       'UGDS_HISP', 'UGDS_ASIAN', 'UGDS_AIAN', 'UGDS_NHPI', 'UGDS_2MOR',
       'UGDS_NRA', 'UGDS_UNKN', 'PPTUG_EF', 'NPT4_PUB', 'NPT4_PRIV', 'PCTPELL',
       'RET_FT4', 'RET_FTL4', 'RET_PT4', 'RET_PTL4', 'PCTFLOAN', 'UG25abv',
       'GRAD_DEBT_MDN_SUPP', 'GRAD_DEBT_MDN10YR_SUPP', 'RPY_3YR_RT_SUPP',
       'C150_4_POOLED_SUPP', 'C200_L4_POOLED_SUPP', 'md_earn_wne_p10',
       'gt_25k_p6', 'gt_25k_05_p6'],
      dtype='object')

In [5]:
# We need to create some treatment variable columns

# Binary columns indicating if a university is public, private not for profit, or private for profit
train_data['CONTROL_PUBLIC'] = (train_data['CONTROL'] == 1)
train_data['CONTROL_PRIVATE_NP'] = (train_data['CONTROL'] == 2)
train_data['CONTROL_PRIVATE_FP'] = (train_data['CONTROL'] == 3)

# What about attending a big or small school? Denote by 1 if a university has a population
# above the average (~2300)
train_data['UGDS_AVG_THRESH'] = (train_data['UGDS'] >= train_data['UGDS'].mean())

# What about attending a really big school (>10000 undergrads)? Or a really small
# school (<1000) undergrads?
train_data['UGDS_10000_THRESH'] = (train_data['UGDS'] >= 10000)
train_data['UGDS_1000_THRESH'] = (train_data['UGDS'] < 1000)

train_data['UGDS_25abv_AVG_THRESH'] = (train_data['UG25abv'] >= train_data['UG25abv'].mean())

In [6]:
train_data['gt_25k_p6_BIN'] = (train_data['gt_25k_p6'] >= .5)
train_data['gt_25k_p6_BIN'] = train_data['gt_25k_p6_BIN'].astype(int)

In [7]:
ols_results = smf.ols('md_earn_wne_p10 ~ CONTROL_PUBLIC + CONTROL_PRIVATE_NP + CONTROL_PRIVATE_FP + UGDS_AVG_THRESH + UGDS_10000_THRESH + UGDS_1000_THRESH + UGDS_25abv_AVG_THRESH', data=train_data).fit()

In [8]:
ols_results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        md_earn_wne_p10   R-squared:                       0.202
Model:                            OLS   Adj. R-squared:                  0.201
Method:                 Least Squares   F-statistic:                     190.4
Date:                Wed, 13 Mar 2019   Prob (F-statistic):          7.59e-217
Time:                        20:39:45   Log-Likelihood:                -48954.
No. Observations:                4515   AIC:                         9.792e+04
Df Residuals:                    4508   BIC:                         9.797e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
=================================================================================================
                                    coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------
Intercept                      2.767e+04    308.449     89.695      0.000    2.71e+04    2.83e+04
CONTROL_PUBLIC[T.True]         6078.5205    339.924     17.882      0.000    5412.103    6744.938
CONTROL_PRIVATE_NP[T.True]     1.572e+04    300.153     52.385      0.000    1.51e+04    1.63e+04
CONTROL_PRIVATE_FP[T.True]     5864.2357    344.089     17.043      0.000    5189.653    6538.818
UGDS_AVG_THRESH[T.True]         650.3000    609.983      1.066      0.286    -545.565    1846.165
UGDS_10000_THRESH[T.True]      4053.5627    803.859      5.043      0.000    2477.604    5629.521
UGDS_1000_THRESH[T.True]      -7755.9927    535.247    -14.490      0.000   -8805.340   -6706.645
UGDS_25abv_AVG_THRESH[T.True]  1536.6095    428.033      3.590      0.000     697.454    2375.765
==============================================================================
Omnibus:                     3924.811   Durbin-Watson:                   1.986
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           296841.806
Skew:                           3.758   Prob(JB):                         0.00
Kurtosis:                      42.005   Cond. No.                     1.26e+15
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 5.58e-27. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

In [9]:
log_results = smf.logit('gt_25k_p6_BIN ~ CONTROL_PUBLIC + CONTROL_PRIVATE_NP + CONTROL_PRIVATE_FP + UGDS_AVG_THRESH + UGDS_10000_THRESH + UGDS_1000_THRESH + UGDS_25abv_AVG_THRESH', data=train_data).fit()

Optimization terminated successfully.
         Current function value: 0.587164
         Iterations 5


In [10]:
log_results.summary()

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/statsmodels/base/model.py:1092: RuntimeWarning: invalid value encountered in sqrt
  bse_ = np.sqrt(np.diag(self.cov_params()))
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/scipy/stats/_distn_infrastructure.py:877: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/scipy/stats/_distn_infrastructure.py:877: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/scipy/stats/_distn_infrastructure.py:1831: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:          gt_25k_p6_BIN   No. Observations:                 6243
Model:                          Logit   Df Residuals:                     6236
Method:                           MLE   Df Model:                            6
Date:                Wed, 13 Mar 2019   Pseudo R-squ.:                  0.1246
Time:                        20:39:45   Log-Likelihood:                -3665.7
converged:                       True   LL-Null:                       -4187.4
                                        LLR p-value:                3.715e-222
=================================================================================================
                                    coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------------
Intercept                        -0.0608        nan        nan        nan         nan         nan
CONTROL_PUBLIC[T.True]           -0.1133        nan        nan        nan         nan         nan
CONTROL_PRIVATE_NP[T.True]        0.5853        nan        nan        nan         nan         nan
CONTROL_PRIVATE_FP[T.True]       -0.5328        nan        nan        nan         nan         nan
UGDS_AVG_THRESH[T.True]           0.5299      0.097      5.469      0.000       0.340       0.720
UGDS_10000_THRESH[T.True]         0.5992      0.143      4.203      0.000       0.320       0.879
UGDS_1000_THRESH[T.True]         -1.0069      0.077    -13.027      0.000      -1.158      -0.855
UGDS_25abv_AVG_THRESH[T.True]     0.3578      0.067      5.378      0.000       0.227       0.488
=================================================================================================
"""

In [14]:
smf.ols('md_earn_wne_p10 ~ CONTROL_PUBLIC*UGDS_10000_THRESH', data=train_data).fit().summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        md_earn_wne_p10   R-squared:                       0.016
Model:                            OLS   Adj. R-squared:                  0.016
Method:                 Least Squares   F-statistic:                     25.22
Date:                Wed, 13 Mar 2019   Prob (F-statistic):           3.53e-16
Time:                        20:40:55   Log-Likelihood:                -49427.
No. Observations:                4515   AIC:                         9.886e+04
Df Residuals:                    4511   BIC:                         9.889e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
====================================================================================================================
                                                       coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------------------
Intercept                                         3.275e+04    250.971    130.475      0.000    3.23e+04    3.32e+04
CONTROL_PUBLIC[T.True]                            -151.9547    472.702     -0.321      0.748   -1078.682     774.772
UGDS_10000_THRESH[T.True]                         1.159e+04   2522.644      4.594      0.000    6642.296    1.65e+04
CONTROL_PUBLIC[T.True]:UGDS_10000_THRESH[T.True] -5400.0061   2672.436     -2.021      0.043   -1.06e+04    -160.721
==============================================================================
Omnibus:                     3544.106   Durbin-Watson:                   2.002
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           193177.645
Skew:                           3.287   Prob(JB):                         0.00
Kurtosis:                      34.363   Cond. No.                         19.0
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""